In [ ]:
!pip install sentence_transformers

In [3]:
from sentence_transformers import SentenceTransformer, util
import math

In [4]:
embedder1 = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
embedder2 = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [7]:
test_cases = [['''In The Sopranos, the mob is besieged as much by inner infidelity as it is by the federal government. Early in the series, the greatest threat to Tony's Family is his own biological family. One of his closest associates turns witness for the FBI, his mother colludes with his uncle to contract a hit on Tony, and his kids click through Web sites that track the federal crackdown in Tony's gangland.''','''In the first season of The Sopranos, Tony Soprano’s mobster activities are more threatened by members of his biological family than by agents of the federal government. This familial betrayal is multi-pronged. Tony’s closest friend and associate is an FBI informant, his mother and uncle are conspiring to have him killed, and his children are surfing the Web for information about his activities.'''],
             ['''William Shakespeare was born in Stratford-on-Avon in April (probably April 23), 1564. His father was a citizen of some prominence who became an alderman and bailiff, but who later suffered financial reverses. Shakespeare presumably attended the Stratford grammar school, where he could have acquired a respectable knowledge of Latin, but he did not proceed to Oxford or Cambridge. There are legends about Shakespeare’s youth but no documented facts.''','''William Shakespeare was born in 1564 in Stratford-on-Avon. His father, a respected alderman, and bailiff was an affluent community member but later lost his financial security. Experts suspect that Shakespeare went to the Stratford grammar school where he probably obtained a command of the Latin language, however, since there are no documented facts about his childhood, scholars rely on rumors and stories believed to be historically accurate. They do know that he did not continue his education at Oxford or Cambridge.'''],
             ['''Exercise can help a lot in alleviating stress – that is a known fact. Exercise is a good way of reducing stress, and cardiovascular exercise is recommended for about 15 to 30 minutes, thrice or four times a week. Several studies have indicated the effects of exercise in handling stress. The activity can release endorphins to the bloodstream.''','''Exercise is a good way to get rid of stress. It is also a perfect way of lowering stress levels, while cardiovascular exercises that can be done three or four times a week for about 15 to 30 minutes is highly suggested. Research shows that there are positive effects of exercise in dealing with stress. It can help in releasing endorphins in the body.'''],
             ['''Dogs can provide great assistance to both children and elderly people in their daily activities. Since dogs are active pets, they can also prove to be the perfect buddies during exercise. There are different types of dogs that you can choose from to make as pets.''','''Dogs offer help to the elderly and children in their daily life. Dogs are active pets, providing to be great exercise buddies. There are various breeds of dogs that you can select from for your pets.'''],
             ['''Police have arrested a 28-year-old man who they say broke into an Austin business, stole a couple of sausages and fell asleep.''','''A 28 year old man in Austin, Texas didn’t break into a business and run, like most criminals. Instead, he took several sausages then simply fell asleep inside the business.'''],
             ['''Police in New Jersey say a man told officers he fled the scene of an auto accident because he ‘didn’t want to deal with his girlfriend yelling at him.''','''A man in New Jersey seemed to think leaving the scene of a car crash and being arrested was a better option than listening to his girlfriend yell. He told the police he fled because he didn’t want to listen to her yell at him.'''],
             ['''The video of Topen’s dancing has racked up more than 400,000 views since it was posted on YouTube last week, and the plumber says he’s already been approached in public for his autograph.''','''Even though the YouTube video of the dancing plumber was only posted last week, it has already had more than 400,000 views. Topen has become an almost instant celebrity as strangers have even asked him for autographs.'''],
             ['''According to Heat magazine, Miley has a list of intense rules for her men-to-be while out on dates. Apparently her assistant arranges what the guy must wear, do, and talk about on the date. She’s also not into flowers, so he’s banned from bringing her those.''','''According to Heat magazine, Miley Cyrus has a list of rules for her men on dates. Her assistant arranges what guys wear and what they talk about. She doesn’t like flowers, either''']
             ]

In [8]:
text1 = test_cases[0][0]
text2 = test_cases[0][1]

In [9]:
text1 = ' '.join(text1.split())
text2 = ' '.join(text2.split())

In [10]:
# Returns scores out of 10
def return_score(text1, text2):
    embedding1, embedding2 = embedder1.encode([text1, text2], convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    score1 = cosine_scores.item()
    score1 = round((90 - 180*math.acos(score1)/math.pi)*2/9)/2
    
    embedding1, embedding2 = embedder2.encode([text1, text2], convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    score2 = cosine_scores.item()
    score2 = round(score2*20)/2
    
    if score2 - score1 > 2.5:
        return abs(score1-2)
    return score2

In [11]:
return_score(text1, text2)

9.5

In [12]:
for _ in test_cases:
    print(*_, sep='\n\n\n')
    print('\n\n\n----------------------------\n\n\n')

In The Sopranos, the mob is besieged as much by inner infidelity as it is by the federal government. Early in the series, the greatest threat to Tony's Family is his own biological family. One of his closest associates turns witness for the FBI, his mother colludes with his uncle to contract a hit on Tony, and his kids click through Web sites that track the federal crackdown in Tony's gangland.


In the first season of The Sopranos, Tony Soprano’s mobster activities are more threatened by members of his biological family than by agents of the federal government. This familial betrayal is multi-pronged. Tony’s closest friend and associate is an FBI informant, his mother and uncle are conspiring to have him killed, and his children are surfing the Web for information about his activities.



----------------------------



William Shakespeare was born in Stratford-on-Avon in April (probably April 23), 1564. His father was a citizen of some prominence who became an alderman and bailiff, b

In [13]:
for _ in test_cases:
    text1, text2 = _
    print(return_score(text1,text2))

9.5
9.0
9.0
9.5
3.0
3.5
9.5
9.5
